# API idealista

Use the idealista.com API to run queries https://api.idealista.com/

## 0. Import libraries

In [1]:
import requests
# Authentication available in Requests
from requests.auth import HTTPBasicAuth
import urllib
import os
import json
import pandas as pd
import argparse
import time

## 1. API credentials

In [2]:
# Sent by idealista
API_KEY= 
SECRET_KEY=

IMP: Access is ``free`` to a maximum of ``100 req/month`` and it’s limited by ``1 req/sec``. 

## 2. OAuth 2.0 Authentication

In [3]:
def get_oauth_token(key, secret):
    '''
    Function to get oauth2 token from the API Key and Secret Key provided by idealista
    '''
    
    # Resource URL
    oauth_url = 'https://api.idealista.com/oauth/token'
    # Parameters
    body = {'grant_type': 'client_credentials'}
    # Response
    r = requests.post(oauth_url,
                      auth=HTTPBasicAuth(key, secret),
                      data=body)
#     print(r.text)
    return r.text

In [4]:
token_json = get_oauth_token(API_KEY, SECRET_KEY)
token_response = json.loads(token_json)
# print(token_response)
TOKEN_VALUE = token_response["access_token"]
#  print("Token: " + TOKEN_VALUE)

## 3. Property search

In [5]:
# Resource URL
base_url =  'https://api.idealista.com/3.5/es/search'

In [6]:
# Example of URLs --> Used to verify if the API works good
API_URL = 'https://api.idealista.com/3/es/search?locale=es&maxItems=20&numPage=1&operation=sale&order=publicationDate&propertyType=garages&sort=desc&apikey={api_key}&t=1&language=es&bankOffer=true&locationId=0-EU-ES-28'

Search for all the **houses for rent** in Barcelona, in a **area** with a **radius of 7.5 km from ironhack**

``Api version: 3.5``

#### · Parameters

In [7]:
# Location (Barcelona)
center='41.3993928,2.1991617' # IronHack
distance= 7500 # distance to center(m) (radius=IronHack-Fira de Barcelona. aprox=7.5 km)
country='es'

# Response
max_items=20 # Items per page --> 50 as maximun allowed
num_page=1
# total=500 # Total items

#### · Type of property and operation(sale/rent)

Type of property --> Homes for rent


In [8]:
# Property type
property_type = 'homes'
# Operation
operation = 'rent'

## 3.1 First page

In [9]:
def url_creator(center, country, max_items, num_page, distance, property_type, operation):
    '''
    Function to create the URL of the first page of the API
    '''
    
    # Resource URL
    base_url = ('https://api.idealista.com/3.5/%s/search?')%(country)   
    
    # Parameters
    query = (('center=%s'+'&country=%s'+'&maxItems=%s'+'&numPage=%s'+'&distance=%s'+'&propertyType=%s'+'&operation=%s')%(center, country, max_items, num_page, distance, property_type, operation))
    
    return base_url+query

In [10]:
# 
URL_HOMES_RENT = url_creator(center, country, max_items, num_page, distance, property_type, operation)
URL_HOMES_RENT

'https://api.idealista.com/3.5/es/search?center=41.3993928,2.1991617&country=es&maxItems=20&numPage=1&distance=7500&propertyType=homes&operation=rent'

In [11]:
def search_api(url, token):
    '''
    Function to run the search using the API and the token
    '''

    # Resource URL
    api_url = url
    # Headers
    headers = {'Authorization': 'Bearer ' + token}
    # Response
    r = requests.post(api_url,
                      headers=headers)
#     print(r.text)
    return r.text

In [12]:
review = search_api(API_URL, TOKEN_VALUE)
review

'{"message":"internal server error","httpStatus":500}'

I made ``too many requests`` \o/ --> **Idealista blocks the API!**


Waiting for new credentials to use it

In [13]:
def property_json(url):
    '''
    Creating the .json file
    '''
    
    response = requests.get(url)
    # Access is limited by 1 req/sec.
    time.sleep(3)
    
    return response.json()

In [14]:
homes_rent = property_json(URL_HOMES_RENT)
homes_rent

{'error': 'unauthorized',
 'error_description': 'authentication data was not found'}

## 3.2 Investigation --> total pages

In [15]:
# Items for page
len(homes_rent['elementList'])

KeyError: 'elementList'

#### · Pagination+Resumen


In [ ]:
total = homes_rent['total']
total_pages = homes_rent['totalPages']
actual_page = homes_rent['actualPage']
summary = homes_rent['summary']

print(f'Total: {total}')
print(f'Total pages: {total_pages}')
print(f'Actual page: {actual_page}')
print(f'Summery: {summary}')

It's NOT possible to visit all the pagesfor the restrictions imposed by Idealista!

## 3.3 All (some) pages

ATTENTION: ``Restrictions to the API``

--> IMP: Access is ``free`` to a ``maximum of 100 req/month`` and it’s limited by ``1 req/sec``.

In [ ]:
def all_URLs(max=3): 
    '''
    Function to iterate through the pages of the API
    max 3 --> max iteration of default to avoid overcoming the restrictions imposed by Idealista 
    '''
    
    all_urls = []   
    # Resource URL
    base_url = ('https://api.idealista.com/3.5/%s/search?')%(country)   

    for page in range(1,max):
        # Parameters
        query = (('center=%s'+'&country=%s'+'&distance=%s'+'&maxItems=%s'+'&numPage=%s'+'&propertyType=%s'+'&operation=%s')%(center, country, distance, max_items, page, property_type, operation))
        url = base_url+query
        all_urls.append(url)
        print (all_urls)
        
    return all_urls

In [ ]:
# 3 pages
all_URLs = all_URLs()
all_URLs

In [ ]:
# First page
all_URLs[0]

In [ ]:
# Last page
all_URLs[-1]

In [ ]:
# I don't know if it works - CHECK!

In [ ]:
def page_selection():
'''
Function that create .json of all pages analyzed
'''

    homes_rent = []
        for i in all_URLs:
            homes_rent.append(property_json(i))
    
    # Dataframe
    homesFORrent = json_normalize(homes_rent)
    return homesFORrent.head()

In [ ]:
homes_rent_BCN = page_selection(all_URLs)
homes_rent_BCN

## 3.2 Export search results

In [ ]:
# homes_BCN = 'homes_rent_BCN_'+time.strftime('%d-%m-%Y_%H-%M')+'.json'

In [ ]:
# with open("output/',homes_BCN 'w') as export:
#     json.dump(homes_rent_BCN, export)

In [ ]:
# Share data between Jupyter Notebooks
# %store homes_BCN

## 4. Extract information with Pandas

--> Coming soon! \o/